<a href="https://colab.research.google.com/github/subod4/NLP/blob/main/Text_Summarizer_abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Abstractive Summary

In [ ]:
!pip install transformers datasets torch rouge-score

Load Data Set and Preprocess

In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"].shuffle(seed=42).select(range(10000))
val_data = dataset["validation"].shuffle(seed=42).select(range(2000))

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_data.map(preprocess_function, batched=True)
val_dataset = val_data.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")


Train the Model

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()


Save The fine tuned Model

In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


Download the model

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("saved_model", 'zip', "./saved_model")
files.download("saved_model.zip")


To load the saved model for inference:

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model_path = "./saved_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def summarize_text(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=10, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Test the model
print(summarize_text("Your input text here..."))


Display with Gradio

In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(label="Input Text", lines=10, placeholder="Enter text here..."),
    outputs=gr.Textbox(label="Summary", lines=5),
    title="Fine-Tuned T5 Summarizer",
    description="This summarizer uses a fine-tuned T5 model for text summarization.",
)

interface.launch()
